In [1]:
from fastai.vision.all import *

/home/haru/mambaforge/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libc10_cuda.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [2]:
path = untar_data(URLs.MNIST_SAMPLE)
Path.BASE_PATH = path

In [3]:
three_trained = torch.stack([tensor(Image.open(o)) for o in (path/'train'/'3').ls()])/255.
seven_trained = torch.stack([tensor(Image.open(o)) for o in (path/'train'/'7').ls()])/255.
three_valid = torch.stack([tensor(Image.open(o)) for o in (path/'valid'/'3').ls()])/255.
seven_valid = torch.stack([tensor(Image.open(o)) for o in (path/'valid'/'7').ls()])/255.

X_train = torch.cat([three_trained, seven_trained]).reshape(-1, 28*28)
y_train = tensor([1]*len(three_trained) + [0]*len(seven_trained)).reshape(-1, 1)
train_dset = list(zip(X_train, y_train))

X_valid = torch.cat([three_valid, seven_valid]).reshape(-1, 28*28)
y_valid = tensor([1]*len(three_valid) + [0]*len(seven_valid)).reshape(-1, 1)
valid_dset = list(zip(X_valid, y_valid))

In [4]:
dl = DataLoader(train_dset, batch_size=256, shuffle=True)
dl_valid = DataLoader(valid_dset, batch_size=256)

In [5]:
dls = DataLoaders(dl, dl_valid)

In [34]:
def dl(p, shuffle=False):
    def f(x):
        return [tensor(Image.open(o)) for o in (x).ls()]
    X3 = torch.stack(f(path/p/'3'))
    X7 = torch.stack(f(path/p/'7'))
    y = tensor([1]*len(X3) + [0]*len(X7)).reshape(-1, 1)
    y1 = tensor([1]*len(X3) + [0]*len(X7))
    X = torch.cat([X3, X7]).view(-1, 28*28)/255.
    ds = list(zip(X, y))
    ds1 = list(zip(X, y1))
    for i, (a, b) in enumerate(zip(ds, ds1)):
        if not (a[0]==b[0]).all():
            print(i)
        if not (a[1]==b[1]).all():
            print(i)
    dl = DataLoader(ds, batch_size=256, shuffle=shuffle)
    return dl

dls = DataLoaders(dl('train', shuffle=True), dl('valid'))

In [7]:
def mnist_loss(pred, targ):
    pred = pred.sigmoid()
    return torch.where(targ==1, 1 - pred, pred).mean()

def batch_accuracy(pred, targ):
    return ((pred>0.)==targ).float().mean()

In [35]:
model = nn.Sequential(
    nn.Linear(28*28, 30),
    nn.ReLU(),
    nn.Linear(30, 1))
learn = Learner(dls, model, loss_func=mnist_loss, metrics=batch_accuracy)
learn.fit(10)

epoch,train_loss,valid_loss,batch_accuracy,time
0,0.158316,0.061910,0.969087,00:00
1,0.072575,0.038065,0.974485,00:00
2,0.043166,0.029610,0.977920,00:00
3,0.029809,0.025430,0.979882,00:00
4,0.022952,0.022230,0.983317,00:00
5,0.019190,0.020424,0.984298,00:00
6,0.016607,0.018710,0.984789,00:00
7,0.015253,0.018397,0.984298,00:00
8,0.014054,0.017153,0.984789,00:00
9,0.013139,0.016817,0.984789,00:00
